# Using ML to predict the Sale Price of Bulldozers 

<span style="color:violet">Special thanks to Zero To Mastery Academy (https://academy.zerotomastery.io) and to its instructor Daniel Bourke.</span>

## <span style="color:red">1. Problem Definition
Prediction of future sale price of bulldozers, taking into account characteristics and examples of how much similar bulldozers have been sold for.
## <span style="color:red">2. Data
##### Bluebook for Bulldozers
Link to data & details: https://www.kaggle.com/c/bluebook-for-bulldozers/data
<p></p>

##### Background information on CSVs:
* Train.csv is the training set, which contains data through the end of 2011.
* Valid.csv is the validation set, which contains data from January 1, 2012 - April 30, 2012 You make predictions on this set throughout the majority of the competition. Your score on this set is used to create the public leaderboard.
* Test.csv is the test set, which won't be released until the last week of the competition. It contains data from May 1, 2012 - November 2012. Your score on the test set determines your final rank for the competition.
## <span style="color:red">3. Evaluation
The evaluation metric for this competition is the RMSLE (root mean squared log error) between the actual and predicted auction prices.
For more see: www.kaggle.com/competitions/bluebook-for-bulldozers/overview/evaluation.
<p></p>
(The goal will be to minimize the RMSLE.)
<p></p>

## <span style="color:red">4. Features
A data dictionary is provided by Kaggle. See: https://www.kaggle.com/competitions/bluebook-for-bulldozers/data.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn

In [ ]:
df = pd.read_csv("data/TrainAndValid.csv",low_memory=False)
df.head()

In [ ]:
df.info()

In [ ]:
df.isna().sum()

In [ ]:
fig, ax = plt.subplots(figsize=(20,6))
ax.scatter(df["saledate"][:1000],df["SalePrice"][:1000]);

In [ ]:
df["SalePrice"].plot.hist();

### Parsing dates
The original datatype of each value in the "saledate" column is of type "object" (= str) and needs to be converted into datetime object.

In [ ]:
df["saledate"][:3]

In [ ]:
df["saledate"].dtype

In [ ]:
type(df["saledate"][0])

In [ ]:
# re-load of data set (TrainsAndValid.csv)
df = pd.read_csv("data/TrainAndValid.csv", low_memory=False, parse_dates=["saledate"])

In [ ]:
df["saledate"][:3]

In [ ]:
df["saledate"].dtype

In [ ]:
type(df["saledate"][0])

### Sorting DataFrame by "saledate" column

In [ ]:
df.sort_values(by="saledate",inplace=True,ascending=True)

In [ ]:
df["saledate"].head(10)

### Creating a backup of modified DataFrame (by copying df)

In [ ]:
df_tmp = df.copy()

### Adding datetime params for "saledate" column
Each value in the "saledate" column will be extracted and saved in its own column. Afterwards the "saledate" column will be deleted.

In [ ]:
df_tmp["saleYear"] = df_tmp["saledate"].dt.year
df_tmp["saleMonth"] = df_tmp["saledate"].dt.month
df_tmp["saleDay"] = df_tmp["saledate"].dt.day
df_tmp["saleDayOfWeek"] = df_tmp["saledate"].dt.dayofweek
df_tmp["saleDayOfYear"] = df_tmp["saledate"].dt.dayofyear

In [ ]:
df_tmp.drop(columns=["saledate"],inplace=True)

## <span style="color:red">5. Modelling

In [ ]:
from sklearn.ensemble import RandomForestRegressor

model = RandomForestRegressor(n_jobs=-1,
                              random_state=42)

# next step would be to fit the model:
# model.fit(df_tmp.drop("SalePrice",axis=1),df_tmp["SalePrice"])

# in the current state the fitting will provoke a ValueError,
# since 44 columns in the data set are not of numeric data types,
# hence, (the commented-out code will not be executed and)
# the non-numeric data types have to be converted first into a processible, i.e. numeric data type

In [ ]:
df_tmp.isna().sum()

In [ ]:
for label, content in df_tmp.items():
    if pd.api.types.is_object_dtype(content):
        print(label)

### Converting Non-Numeric Objects (strings & NaN) to Categorical Data
In order to use the data for ML model(s), it has to be converted from String- or NaN-objects to categorical data. Hence, the goal of the following modification is to change all non-numeric objects into numeric objects.
<p></p>
An overview of how to search for data types with pandas is to be found here: <a href="https://pandas.pydata.org/docs/reference/arrays.html">pandas documentation</a> (see the subchapter "Utilities").


In [ ]:
for label, content in df_tmp.items():
    if pd.api.types.is_object_dtype(content):
        df_tmp[label] = content.astype("category").cat.as_ordered()

In [ ]:
df_tmp.info()

In [ ]:
# Portays missing data per column in percent (%)
df_tmp.isna().sum() / len(df_tmp) * 100

### Saving processed data

In [ ]:
df_tmp.to_csv("data/train_tmp.csv", index=False)

In [ ]:
#df_tmp = pd.read_csv("data/train_tmp.csv", low_memory=False)
#df_tmp.head()

In [ ]:
#df_tmp.info()

In [ ]:
# since the goal is to predict the sale price for bulldozers for the year 2012
# based on data from 1989 to 2011, two DataFrames are created
# (one only with the sale year 2012 and one excluding the sale year 2012)

df_train = df_tmp[df_tmp["saleYear"] != 2012]
df_val = df_tmp[df_tmp["saleYear"] == 2012]

len(df_train), len(df_val)

### Filling missing numeric values with the median of the existing ones

###### filling df_train

In [ ]:
count = 0
for label, content in df_train.items():
    if pd.api.types.is_numeric_dtype(content):
        if pd.isnull(content).sum():
            print(label)
            count+=pd.isnull(content).sum()
if count > 0:
    print(f"count: {count}")
else:
    print(f"count: {count} (no NaN / null-values in the data set)")

In [ ]:
for label, content in df_train.items():
    if pd.api.types.is_numeric_dtype(content):
        if pd.isnull(content).sum():
            #df_train[label+"_is_missing"] = pd.isnull(content)
            df_train[label] = content.fillna(content.median())

In [ ]:
count = 0
for label, content in df_train.items():
    if pd.api.types.is_numeric_dtype(content):
        if pd.isnull(content).sum():
            print(label)
            count+=pd.isnull(content).sum()
if count > 0:
    print(f"count: {count}")
else:
    print(f"count: {count} (no NaN / null-values in the data set)")

###### filling df_val

In [ ]:
count = 0
for label, content in df_val.items():
    if pd.api.types.is_numeric_dtype(content):
        if pd.isnull(content).sum():
            print(label)
            count+=pd.isnull(content).sum()
if count > 0:
    print(f"count: {count}")
else:
    print(f"count: {count} (no NaN / null-values in the data set)")

In [ ]:
for label, content in df_val.items():
    if pd.api.types.is_numeric_dtype(content):
        if pd.isnull(content).sum():
            #df_val[label+"_is_missing"] = pd.isnull(content)
            df_val[label] = content.fillna(content.median())

In [ ]:
count = 0
for label, content in df_val.items():
    if pd.api.types.is_numeric_dtype(content):
        if pd.isnull(content).sum():
            print(label)
            count+=pd.isnull(content).sum()
if count > 0:
    print(f"count: {count}")
else:
    print(f"count: {count} (no NaN / null-values in the data set)")

### Filling and turning categorical values into numeric ones

###### fill df_train

In [ ]:
count = 0
for label, content in df_train.items():
    if not pd.api.types.is_numeric_dtype(content):
        if pd.isnull(content).sum():
            print(label)
            count+=pd.isnull(content).sum()
if count > 0:
    print(f"count: {count}")
else:
    print("no NaN / null-values in the data set")

In [ ]:
for label, content in df_train.items():
    if not pd.api.types.is_numeric_dtype(content):
        #df_train[label+"_is_missing"] = pd.isnull(content)
        df_train[label] = pd.Categorical(content).codes+1 # the +1 is turning the default -1 (= NaN) to 0

In [ ]:
count = 0
for label, content in df_train.items():
    if not pd.api.types.is_numeric_dtype(content):
        if pd.isnull(content).sum():
            print(label)
            count+=pd.isnull(content).sum()
if count > 0:
    print(f"count: {count}")
else:
    print("no NaN / null-values in the data set")

###### fill df_val 

In [ ]:
count = 0
for label, content in df_val.items():
    if not pd.api.types.is_numeric_dtype(content):
        if pd.isnull(content).sum():
            print(label)
            count+=pd.isnull(content).sum()
if count > 0:
    print(f"count: {count}")
else:
    print("no NaN / null-values in the data set")

In [ ]:
for label, content in df_val.items():
    if not pd.api.types.is_numeric_dtype(content):
        #df_val[label+"_is_missing"] = pd.isnull(content)
        df_val[label] = pd.Categorical(content).codes+1 # the +1 is turning the default -1 (= NaN) to 0

In [ ]:
count = 0
for label, content in df_val.items():
    if not pd.api.types.is_numeric_dtype(content):
        if pd.isnull(content).sum():
            print(label)
            count+=pd.isnull(content).sum()
if count > 0:
    print(count)
else:
    print(f"count: {count} (no NaN / null-values in the data set)")

In [ ]:
X_train, y_train = df_train.drop("SalePrice",axis=1), df_train["SalePrice"]
X_valid, y_valid = df_val.drop("SalePrice",axis=1), df_val["SalePrice"]

X_train.shape, y_train.shape, X_valid.shape, y_valid.shape

### Model score

In [ ]:
%%time
# instantiate model
model = RandomForestRegressor(n_jobs=-1,
                              random_state=42)
# fit model
model.fit(X_train,y_train)

In [ ]:
# tested data is based on the same training data, which results in high score
# (nearly 99%), but that means that no generalization can be drawn out of this

model.score(X_train,y_train)

### Build an evaluation function

In [ ]:
from sklearn.metrics import mean_squared_log_error, mean_absolute_error

def rmsle(y_true, y_pred):
    return np.sqrt(mean_squared_log_error(y_true, y_pred))

def show_scores(model):
    train_pred = model.predict(X_train)
    val_pred = model.predict(X_valid)
    scores = {"Training MAE": mean_absolute_error(y_train, train_pred),
              "Valid MAE": mean_absolute_error(y_valid, val_pred),
              "Training RMSLE": rmsle(y_train, train_pred),
              "Valid RMSLE": rmsle(y_valid, val_pred),
              "Training R^2": model.score(X_train, y_train),
              "Valid R^2": model.score(X_valid, y_valid)}
    return scores

In [ ]:
show_scores(model)

### Hyperparameter tuning with RandomizedSearchCV

In [ ]:
%%time
from sklearn.model_selection import RandomizedSearchCV

rf_grid = {"n_estimators":np.arange(10,100,10),
           "max_depth":[None,3,5.10],
           "min_samples_split":np.arange(2,20,2),
           "min_samples_leaf":np.arange(1,20,2),
           "max_features":[0.5,1,"sqrt","auto"],
           "max_samples":[200000]}

rs_model = RandomizedSearchCV(RandomForestRegressor(n_jobs=-1,
                                                    random_state=42),
                              param_distributions=rf_grid,
                              n_iter=100,
                              cv=5,
                              verbose=True)
rs_model.fit(X_train,y_train)

In [ ]:
# best model hyperparams
rs_model.best_params_

In [ ]:
show_scores(rs_model)

In [ ]:
%%time
best_hyperparams_model = RandomForestRegressor(n_estimators=50,
                                               min_samples_split=2,
                                               min_samples_leaf=3,
                                               max_features=0.5,
                                               max_samples=None,
                                               max_depth=None,
                                               random_state=42)
best_hyperparams_model.fit(X_train,y_train)

In [ ]:
show_scores(best_hyperparams_model)

### Make predictions on test data set

In [ ]:
# import of test data set
df_test = pd.read_csv("data/Test.csv",low_memory=False,parse_dates=["saledate"])
df_test.head()

In [ ]:
# test data set is given as argument to best hyperparam model
test_pred = best_hyperparams_model.predict(df_test)

In [ ]:
def preprocess_data(df):
    df["saleYear"] = df["saledate"].dt.year
    df["saleMonth"] = df["saledate"].dt.month
    df["saleDay"] = df["saledate"].dt.day
    df["saleDayOfWeek"] = df["saledate"].dt.dayofweek
    df["saleDayOfYear"] = df["saledate"].dt.dayofyear
    
    df.drop("saledate",
            axis=1,
            inplace=True)
    
    for label, content in df.items():
        if pd.api.types.is_numeric_dtype(content):
            if pd.isnull(content).sum():
                #df[label+"_is_missing"] = pd.isnull(content)
                df[label] = content.fillna(content.median())
    
        if not pd.api.types.is_numeric_dtype(content):
            #df[label+"_is_missing"] = pd.isnull(content)
            df[label] = pd.Categorical(content).codes+1
    
    return df

In [ ]:
df_test = preprocess_data(df_test)
df_test.shape

In [ ]:
X_train.shape

In [ ]:
test_pred = best_hyperparams_model.predict(df_test)

In [ ]:
df_pred = pd.DataFrame()
df_pred["SalesID"] = df_test["SalesID"]
df_pred["SalesPrice"] = test_pred
df_pred

In [ ]:
df_pred.to_csv("data/test_pred.csv",index=False)

In [ ]:
def plot_features(columns,importances,n=20):
    df = (pd.DataFrame({"features":columns,
                        "feature_importances":importances})
          .sort_values("feature_importances",
                       ascending=False)
          .reset_index(drop=True))
    
    fig, ax = plt.subplots()
    ax.barh(df["features"][:n],
            df["feature_importances"][:20])
    ax.set_xlabel("Feature importance")
    ax.set_ylabel("Features")
    ax.invert_yaxis()

In [ ]:
plot_features(X_train.columns,best_hyperparams_model.feature_importances_)